In [1]:
# Itinializing the Jopyter Notebook file VacationPy.ipynb

In [180]:
# Importing relevant dependencies 
import requests
import json
import pandas as pd
import numpy as np
from config import api_key
from pprint import pprint

# Importing gmaps and Google API key
import gmaps
from config import gkey

# Access maps with unique API key
gmaps.configure(api_key=gkey)

In [181]:
# Read in cities from WeatherPy output
city_data=pd.read_csv("../WeatherPy/city_data.csv")
city_data.head()

,City name,Country name,Longitude,Latitude,"Temperature, F","Humidity, %","Cloudiness, %","Wind speed, mph",Hemisphere
0,aasiaat,gl,-53.464459,71.022578,35.53,85,96,2.75,Northern
1,aban,ru,96.046101,57.006436,44.65,50,4,2.71,Northern
2,abu samrah,qa,49.980118,20.223969,95.22,11,0,4.97,Northern
3,acapulco,mx,-107.577970,2.133108,82.83,55,100,0.94,Northern
4,acari,pe,-75.100356,-18.049583,74.16,59,95,2.32,Southern


In [182]:
# Set figure dimensions
figure_layout = {
    'width': '100%',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

In [183]:
# Create a list, which would contain lon, lat coordinates

In [184]:
# Set up gmap with a marker layer containing heatmap of the humidity (%)
locations = city_data[['Latitude', 'Longitude']]
weights = city_data['Humidity, %']
fig = gmaps.figure(layout=figure_layout)
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights, dissipating=False, max_intensity=200,
                                 point_radius=4))
fig


Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

In [198]:
# Narrow down the DataFrame to find your ideal weather condition. For example:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
# Note: Feel free to adjust to your specifications but be sure to limit the number of rows returned by your API requests to a reasonable number.

# Set up weather conditions to filter city_data dataset
max_temp=85.0
min_temp=70.0
max_wind=10.0
max_cloud=0.0

city_data_ideal=city_data[(city_data['Temperature, F']  >  min_temp)  & 
                          (city_data['Temperature, F']  <  max_temp)  &
                          (city_data['Cloudiness, %']   == max_cloud) &
                          (city_data['Wind speed, mph'] <  max_wind)]


In [199]:
len(city_data), len(city_data_ideal)

(1000, 43)

In [200]:
# find the closest hotel within 5000 meters of city coordinates
# first set up one off request
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# params = {
#     "location": "24.799364, -13.952929",  # philadelphia coords
#     "rankby": "distance",
#     "locationbias": "circle:5000",
#     "type": "lodging",
#     "key": gkey,
# }
# #       adding condition to search within 5000 m of lat,lon coordinate  
# #       "locationbias": "circle:5000",
   
# # assemble url and make API request
# response = requests.get(base_url, params=params).json()
# print(response['results'][0]['name'])
    
city_data_ideal['Hotel name']=""    
search_radius=5000    

for index, row in city_data_ideal.iterrows():
    location = f"{city_data_ideal.loc[index, 'Latitude']}, {city_data_ideal.loc[index, 'Longitude']}"
#     print(location)
#     print(location)
    params = {
        "location": str(location),  # philadelphia coords
#         "rankby": "distance",
        "type": "lodging",
#       adding condition to search within 5000 m of lat,lon coordinate  
        "radius": search_radius,
        "key": gkey,
    }
   
    # assemble url and make API request
    response = requests.get(base_url, params=params).json()
    # extract results
    results = response['results']
    
#     pprint(results[0], indent=4)
    
    try:
        print(f"Closest hotel in {city_data_ideal.loc[index, 'City name']} is {results[0]['name']}.")
        
        city_data_ideal.loc[index, 'Hotel name'] = results[0]['name']
        
    except (KeyError, IndexError):
        city_data_ideal.loc[index, 'Hotel name'] = f"No hotel can be found within {params['radius']} meters"
        print(f"------------------ SKIPPING | Missing field/result for {city_data_ideal.loc[index, 'City name']}. ")

<ipython-input-200-9a53433053d1>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  city_data_ideal['Hotel name']=""


Closest hotel in albacete is Casa Rural Tarazona de la Mancha.


C:\Users\troub\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


------------------ SKIPPING | Missing field/result for amboasary. 
------------------ SKIPPING | Missing field/result for ambovombe. 
------------------ SKIPPING | Missing field/result for ano mera. 
------------------ SKIPPING | Missing field/result for beloha. 
------------------ SKIPPING | Missing field/result for cabedelo. 
------------------ SKIPPING | Missing field/result for cap malheureux. 
------------------ SKIPPING | Missing field/result for cascais. 
------------------ SKIPPING | Missing field/result for chissamba. 
------------------ SKIPPING | Missing field/result for colares. 
------------------ SKIPPING | Missing field/result for erzin. 
------------------ SKIPPING | Missing field/result for ewa beach. 
------------------ SKIPPING | Missing field/result for hami. 
------------------ SKIPPING | Missing field/result for jablah. 
------------------ SKIPPING | Missing field/result for kashi. 
------------------ SKIPPING | Missing field/result for kurchum. 
-----------------

In [212]:
city_data_ideal[city_data_ideal['Hotel name']!="No hotel can be found within 5000 meters"].head()

,City name,Country name,Longitude,Latitude,"Temperature, F","Humidity, %","Cloudiness, %","Wind speed, mph",Hemisphere,Hotel name
17,albacete,es,-1.863764,39.276199,72.41,43,0,6.66,Northern,Casa Rural Tarazona de la Mancha
630,palestine,us,-95.489677,31.490991,80.71,34,0,7.19,Northern,Salmon Lake Park
762,san juan,ar,-68.699120,-31.562604,70.74,25,0,2.62,Southern,"Apart Hotel ""Cabañas Solar del Valle"""


In [215]:
city_data_ideal.head()

,City name,Country name,Longitude,Latitude,"Temperature, F","Humidity, %","Cloudiness, %","Wind speed, mph",Hemisphere,Hotel name
17,albacete,es,-1.863764,39.276199,72.41,43,0,6.66,Northern,Casa Rural Tarazona de la Mancha
31,amboasary,mg,46.158640,-24.187486,75.04,75,0,3.73,Southern,No hotel can be found within 5000 meters
33,ambovombe,mg,47.095789,-29.738819,70.57,79,0,3.73,Southern,No hotel can be found within 5000 meters
41,ano mera,gr,25.634546,37.533737,81.23,44,0,5.14,Northern,No hotel can be found within 5000 meters
91,beloha,mg,42.557577,-30.633310,74.97,52,0,3.84,Southern,No hotel can be found within 5000 meters


In [224]:
# Use lat,lon coordinates of select cities to place markers
ideal_locations = city_data_ideal[['Latitude', 'Longitude']]
# city_label_list=(city_data_ideal['Hotel name'] + "\n" +city_data_ideal['City name'] + "\n" + city_data_ideal['Country name']).tolist()
city_label_list=("<b>Hotel</b><br>" + city_data_ideal['Hotel name'] + "<br>" + "<b>City</b><br>" +city_data_ideal['City name'] + "<br>" + "<b>Country</b><br>" + city_data_ideal['Country name']).tolist()

marker_layer = gmaps.marker_layer(ideal_locations, info_box_content=city_label_list)
fig = gmaps.figure(layout=figure_layout)
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights, dissipating=False, max_intensity=200,
point_radius=4))
fig.add_layer(marker_layer)

fig


Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…